Retrieve raw PDB files 

In [1]:
from Bio.PDB.PDBList import PDBList
import os

# Initialize PDBList
pdbl = PDBList()

# Directory where the PDB files will be saved
pdb_directory = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/raw_pdb'
if not os.path.exists(pdb_directory):
    os.makedirs(pdb_directory)

# Path to your pocket PDB files
pocket_pdb_directory = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/dataset/pocket_pdb'

# List to hold the formatted PDB names
pdb_names = []

# Loop through each file in the directory
for filename in os.listdir(pocket_pdb_directory):
    if filename.endswith("_pocket.pdb"):
        # Extract the PDB identifier from the filename
        pdb_id = filename.split("_pocket.pdb")[0]
        
        # Append the formatted name to the list
        pdb_names.append(pdb_id.upper())  # PDB IDs are typically uppercase

for pdb_id in pdb_names:
    # Retrieve PDB file
    pdbl.retrieve_pdb_file(pdb_id, pdir=pdb_directory, file_format='pdb')

    print(f"Downloaded: {pdb_id}")

Downloaded: 5L9G
Downloaded: 1H46
Downloaded: 2PY4
Downloaded: 4JAL
Downloaded: 1NW7
Downloaded: 2P7G
Downloaded: 3QIN
Downloaded: 3GKZ
Downloaded: 1ZZ1
Downloaded: 1DRK
Downloaded: 5HVS
Downloaded: 2V92
Downloaded: 1FKJ
Downloaded: 4DEW
Downloaded: 3BIR
Downloaded: 4W54
Downloaded: 4X1G
Downloaded: 3C9E
Downloaded: 4ZE2
Downloaded: 3B25
Downloaded: 5KR1
Downloaded: 4ALG
Downloaded: 1L5Q
Downloaded: 1YA3
Downloaded: 5ITP
Downloaded: 2F78
Downloaded: 4QJR
Downloaded: 4ZLA
Downloaded: 5FE7
Downloaded: 1CC6
Downloaded: 5LHT
Downloaded: 2QX0
Downloaded: 3VX3
Downloaded: 3HPI
Downloaded: 3T1M
Downloaded: 3F19
Downloaded: 2VC9
Downloaded: 1HS6
Downloaded: 1ZSB
Downloaded: 1UW1
Downloaded: 1RL8
Downloaded: 3DNT
Downloaded: 4F3I
Downloaded: 2UYW
Downloaded: 5KXE
Downloaded: 5E95
Downloaded: 3ARP
Downloaded: 3UJB
Downloaded: 1GX4
Downloaded: 2ZRU
Downloaded: 1K6V
Downloaded: 2IDZ
Downloaded: 4DUF
Downloaded: 1LV8
Downloaded: 1SH9
Downloaded: 4CSD
Downloaded: 2F8I
Downloaded: 3U5L
Downloaded: 4A

Extract features based on the binding pocket.

In [25]:
from Bio.PDB import PDBParser, NeighborSearch, Selection, is_aa 
import pandas as pd 
import subprocess
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import os
import numpy as np
import torch
import torch.nn as nn
import os
import numpy as np
from Bio.PDB.DSSP import DSSP

# Set the working directory
os.chdir('/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project')

class ProteinFeatures:
    
    def __init__(self, pdb_file, pocket_pdb_file=None):
        self.dssp_executable = '/opt/homebrew/bin/mkdssp' 
        self.pocket_pdb_file = pocket_pdb_file
        self.pdb_file = pdb_file
        self.parser = PDBParser(QUIET=True)
        self.amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU', 
                            'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
        self.secondary_structure_codes = ['H', 'B', 'E', 'G', 'I', 'T', 'S', ' ', '.']
        self.pocket_residues = self.load_pocket_residues()
        torch.manual_seed(42)  # Set a fixed seed for random number generation
        self.structure = self.parser.get_structure('protein', self.pdb_file)

    
    
    def extract_sequence(self):
        parser = PDBParser()
        structure = parser.get_structure('protein', self.pdb_file)
        model = structure[0]  # Assuming single model
        chain = model['A']  # Assuming chain A, change as needed
        residues = chain.get_residues()

        aminoacid_counts = {}
        total_residues = 0

        for residue in residues:
            if residue.get_id()[0] == ' ':
                aminoacid = residue.get_resname()
                aminoacid_counts[aminoacid] = aminoacid_counts.get(aminoacid, 0) + 1
                total_residues += 1

        aminoacid_frequencies = {aminoacid: count / total_residues for aminoacid, count in aminoacid_counts.items()}

        return aminoacid_frequencies
    
    def calculate_total_contact(self):
        """Calculate the total contact for each residue based on a distance threshold."""
        atom_list = Selection.unfold_entities(self.structure, 'A')  # Use self.structure
        ns = NeighborSearch(atom_list)
        total_contact_dict = {}

        for chain in self.structure.get_chains():  # Use self.structure
            for residue in chain:
                if not is_aa(residue, standard=True):  # Skip non-amino acid entities, ensure to import is_aa
                    continue
                residue_key = (chain.id, residue.id)
                contacts = ns.search(residue.center_of_mass(), 5.0, level='R')  # Search within 5Å radius
                total_contact_dict[residue_key] = len(contacts) - 1  # Subtract 1 to exclude the residue itself

        return total_contact_dict


    
    def extract_secondary_structure(self):
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure('protein', self.pdb_file)
        model = structure[0]

        # Generate DSSP data using Biopython's DSSP wrapper
        dssp = DSSP(model, self.pdb_file, dssp=self.dssp_executable)

        # Structured data to store the features
        structured_results = {}

        # Iterate over DSSP output to populate structured_results
        for key in dssp.keys():
            res_id = (key[0], key[1][1])  # Chain ID and residue number (ignoring insertion code for simplicity)
            _, _, ss, access, phi, psi = dssp[key][:6]  # Extract the necessary data
            structured_results[res_id] = {
                'secondary_structure': ss,
                'solvent_accessibility': access,
                'phi': phi,
                'psi': psi
            }

        return structured_results
    
    def load_pocket_residues(self):
        pocket_residues_set = set()
        if self.pocket_pdb_file:
            with open(self.pocket_pdb_file, 'r') as file:
                for line in file:
                    if line.startswith('ATOM'):
                        chain = line[21]
                        residue = line[22:26].strip()
                        pocket_residues_set.add((chain, residue))
        return pocket_residues_set
    
    def extract_features(self):
        structure = self.parser.get_structure('protein', self.pdb_file)
        model = structure[0]  # Asumiendo un único modelo para simplificar
        # Inicializar listas para mantener las características extraídas
        dssp_data = self.extract_secondary_structure()
        all_features = []
        for chain in model.get_chains():
            for residue in chain.get_residues():
                if residue.get_id()[0] == ' ':  # Solo residuos estándar
                    residue_id = residue.get_id()
                    pdb_id = self.pdb_file.split('/')[-1].split('.')[0]
                    residue_name = residue.get_resname()
                    secondary_structure = dssp_data.get((chain.id, residue_id[1]), {}).get('secondary_structure', ' ')
                    solvent_accessibility = dssp_data.get((chain.id, residue_id[1]), {}).get('solvent_accessibility', ' ')
                    psi_angle = dssp_data.get((chain.id, residue_id[1]), {}).get('psi', np.nan)
                    phi_angle = dssp_data.get((chain.id, residue_id[1]), {}).get('phi', np.nan)
                    In_pocket = int((chain.id, str(residue_id[1])) in self.pocket_residues)
                    total_contact = self.calculate_total_contact().get((chain.id, residue_id[1]), 0)
        
                    feature_dict = {
                        'PDB_ID': pdb_id,
                        'Chain': chain.id,
                        'Residue_ID': residue_id[1],
                        'Residue_Name': residue_name,
                        'In_Pocket': In_pocket,
                        'Secondary_structure': secondary_structure,
                        'Solvent_accesibility': solvent_accessibility,
                        'Psi_angle': psi_angle,
                        'Phi_angle': phi_angle,
                        'Total_contact': total_contact,
                    }
                    
                    all_features.append(feature_dict)
                    
        return all_features


pdb_file = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/raw_pdb/pdb5u5x.pdb'
pocket_pdb_file = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/dataset/pocket_pdb/5u5x_pocket.pdb'
pf = ProteinFeatures(pdb_file, pocket_pdb_file)
features = pf.extract_features()
print(features)

[{'PDB_ID': 'pdb5u5x', 'Chain': 'A', 'Residue_ID': 4, 'Residue_Name': 'LEU', 'In_Pocket': 1, 'Secondary_structure': '-', 'Solvent_accesibility': 0.8292682926829268, 'Psi_angle': 116.5, 'Phi_angle': 360.0, 'Total_contact': 0}, {'PDB_ID': 'pdb5u5x', 'Chain': 'A', 'Residue_ID': 5, 'Residue_Name': 'VAL', 'In_Pocket': 0, 'Secondary_structure': '-', 'Solvent_accesibility': 0.7253521126760564, 'Psi_angle': 127.6, 'Phi_angle': -115.7, 'Total_contact': 0}, {'PDB_ID': 'pdb5u5x', 'Chain': 'A', 'Residue_ID': 6, 'Residue_Name': 'HIS', 'In_Pocket': 1, 'Secondary_structure': 'E', 'Solvent_accesibility': 0.19021739130434784, 'Psi_angle': 106.3, 'Phi_angle': -116.0, 'Total_contact': 0}, {'PDB_ID': 'pdb5u5x', 'Chain': 'A', 'Residue_ID': 7, 'Residue_Name': 'VAL', 'In_Pocket': 0, 'Secondary_structure': 'E', 'Solvent_accesibility': 0.49295774647887325, 'Psi_angle': 127.5, 'Phi_angle': -93.9, 'Total_contact': 0}, {'PDB_ID': 'pdb5u5x', 'Chain': 'A', 'Residue_ID': 8, 'Residue_Name': 'ALA', 'In_Pocket': 0, 'Se

Prepare the Data

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import h5py

# Asumiendo que features_array es un array de NumPy con tus características extraídas,
# pdb_names es una lista de identificadores PDB correspondientes,
# y affinities es un array de NumPy con la afinidad de cada complejo.

# Convertir pdb_names y affinities a arrays de NumPy para un manejo de datos consistente
pdb_names_array = np.array(pdb_names)
affinities_array = np.array(affinities)

# Primero, divide los datos en conjuntos de entrenamiento + validación y de prueba
X_train_val, X_test, pdb_train_val, pdb_test, aff_train_val, aff_test = train_test_split(
    features_array, pdb_names_array, affinities_array, test_size=0.20, random_state=42)

# Luego, divide el conjunto de entrenamiento + validación en conjuntos de entrenamiento y validación separados
X_train, X_val, pdb_train, pdb_val, aff_train, aff_val = train_test_split(
    X_train_val, pdb_train_val, aff_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Guardar en archivos HDF5
output_path = '../../dataset/'  # Modifica esto según sea necesario

with h5py.File(f'{output_path}training_set.hdf5', 'w') as f_train, \
     h5py.File(f'{output_path}validation_set.hdf5', 'w') as f_val, \
     h5py.File(f'{output_path}test_set.hdf5', 'w') as f_test:

    # Conjunto de entrenamiento
    f_train.create_dataset('features', data=X_train)
    f_train.create_dataset('pdb_ids', data=pdb_train.astype('S'))  # Convertir a bytes
    f_train.create_dataset('affinities', data=aff_train)

    # Conjunto de validación
    f_val.create_dataset('features', data=X_val)
    f_val.create_dataset('pdb_ids', data=pdb_val.astype('S'))  # Convertir a bytes
    f_val.create_dataset('affinities', data=aff_val)

    # Conjunto de prueba
    f_test.create_dataset('features', data=X_test)
    f_test.create_dataset('pdb_ids', data=pdb_test.astype('S'))  # Convertir a bytes
    f_test.create_dataset('affinities', data=aff_test)


Define and Train the Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from utils import progress_bar

# Model Definition
class BindingPocketCNN(nn.Module):
    def __init__(self, num_features, seq_length):
        super(BindingPocketCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=5, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        # Adjust the definition of fc1 according to the sequence length adjusted by the pooling layers
        self.fc1 = nn.Linear(128 * (seq_length // 4), 128)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except for the batch
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Load Data
def load_data(input_path):
    with h5py.File(f'{input_path}/training_set.hdf5', 'r') as f:
        X_train = torch.tensor(f['features'][:], dtype=torch.float)
        y_train = torch.tensor(f['affinities'][:], dtype=torch.float).view(-1, 1)
    return X_train, y_train

# Data Preparation
input_path = '../dataset'  # Adjust as necessary
X_train, y_train = load_data(input_path)
seq_length = X_train.shape[2]  # Assuming features are in dimension 2
num_features = X_train.shape[1]

# Data Splitting (assuming data are already loaded and prepared)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Initialization and Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BindingPocketCNN(num_features=num_features, seq_length=seq_length).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar(i, len(train_loader), 'Loss: %.3f' % (loss.item()))
    
    # Validation step (optional, but recommended)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            progress_bar(i, len(val_loader), 'Val Loss: %.3f' % (val_loss/(i+1)))

    print(f'Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')


ModuleNotFoundError: No module named 'h5py'